In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import holidays

def get_previous_business_day():
    """
    직전 영업일 구하기

    Returns:
        datetime.datetime: 직전 영업일.
    """

    kr_holidays = holidays.KR()
    today = datetime.today().date()

    prev_day = today - timedelta(days=1)
    while prev_day.weekday() >= 5 or prev_day in kr_holidays:
        prev_day -= timedelta(days=1)

    return prev_day

def get_주요지표변동():
    """
    주요지표변동 조회
    """
    pass
    
def get_공시(date: str) -> pd.DataFrame:
    """
    공시 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        pd.DataFrame: 데이터프레임. 컬럼: [시간, 회사명, 공시제목, 링크].
    """

    def is_krx_index_component(td_tag) -> bool:
        """
        해당 종목이 KOSPI200 또는 KOSDAQ150 지수 구성 종목인지 확인
        """
        img_tags = td_tag.find_all('img')
        index_tags = [img.get('alt') for img in img_tags if img.has_attr('alt')]
        return 'KOSPI200' in index_tags or 'KOSDAQ150' in index_tags

    url = "https://kind.krx.co.kr/disclosure/todaydisclosure.do"

    payload = {    
        'method': 'searchTodayDisclosureSub',
        'currentPageSize': '5000',
        'pageIndex': '1',
        'orderMode': '0',
        'orderStat': 'D',
        'marketType': '', 
        'forward': 'todaydisclosure_sub',
        'searchMode': '', 
        'searchCodeType': '', 
        'chose': 'S',
        'todayFlag': 'N',
        'repIsuSrtCd': '',
        'kosdaqSegment': '',
        'selDate': date,
    }

    res = requests.post(url, data=payload)
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.find_all('tr')

    results = []

    for tr in rows:
        tds = tr.find_all('td')

        if len(tds) < 3:
            continue

        if not is_krx_index_component(tds[1]):
            continue

        time = tds[0].text.strip()
        company = tds[1].text.strip()
        title = tds[2].text.strip()
        
        onclick = tds[2].find('a').get('onclick', '')
        match = re.search(r"openDisclsViewer\('(\d+)'", onclick)
        disclosure_id = match.group(1)
        link = f'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={disclosure_id}&docno=&viewerhost=&viewerport='
        
        results.append([time, company, title, link])


    columns = ["시간", "회사명", "공시제목", "링크"]
    return pd.DataFrame(results, columns=columns)

def get_투자경고종목(date: str) -> str:
    """
    투자경고종목 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        str: 투자경고종목.
    """

    url = "https://moc.krx.co.kr/contents/MOC/99/MOC99000001.jspx"

    payload = {
        "market_gubun": "ALL",
        "fromDate": date,
        "toDate": date,
        "pagePath": "/contents/SVL/M/03020200/MOC03020200.jsp",
        "code": "YmD69RMDlLdJjotrXv2qYKFB8V9nBNKG35UprgSdoRZ47Zz/MwrFp84JbilNWZo1b/Lug2VdJQm4mfONjCCfNjRUF6IYVXJgpkKZeyxQ/qt2Ziyg7PkzzcufP2hPFIkroP8zo2lM8Xbh1B+jlRJtIQpE7r9xTp2sNhIQX7myevyuq55WOC+DRARslzH9ldx+",
        "pageFirstCall": "Y"
    }

    res = requests.post(url, data=payload)
    data = json.loads(res.text)

    company_list = [x['kor_isu_nm'] for x in data['DS1']]
    text = ', '.join(company_list) if company_list else '해당없음'
    return text

def get_투자위험종목(date):
    """
    투자위험종목 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        str: 투자위험종목.
    """
        
    url = "https://moc.krx.co.kr/contents/MOC/99/MOC99000001.jspx"

    payload = {
        "market_gubun": "ALL",
        "fromDate": date,
        "toDate": date,
        "pagePath": "/contents/SVL/M/03030200/MOC03030200.jsp",
        "code": "YmD69RMDlLdJjotrXv2qYDJKbIoJPL3rcbQNe5KZ0gaNzAj/pjFcZr/DzeNI0R76PsEoHQodui/1vJKOCFVpT+MwKfYjzRbM0Oit4+IttrphdOQWeKaynWsFPeb5zvmJm2lWk1xUbiHDmgZ/aniEgn4A3zcFVDOhfAMXg6H4Df2Dj7l7hV5e8Lq8Jh3vWyPORpSzuB4rQdTuRS5e1IwCXg==",
        "pageFirstCall": "Y"
    }

    res = requests.post(url, data=payload)
    data = json.loads(res.text)

    company_list = [x['kor_isu_nm'] for x in data['DS1']]
    text = ', '.join(company_list) if company_list else '해당없음'
    return text

def get_주요일정(week):
    """
    주요일정 조회

    Args:
        week (str): 'thisWeek' 또는 'nextWeek' 중 하나.

    Returns:
        str: 주요일정.
    """

    url = "https://kr.investing.com/economic-calendar/Service/getCalendarFilteredData"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
    }

    payload = {
        "timeZone": "88", # 서울 기준
        "country[]": ["5", "11"],  # 미국(5), 한국(11)
        "importance[]": ["3"], # 중요도 3
        "action": "filterEconomicCalendar", # 필수
        "currentTab": week
    }

    res = requests.post(url, headers=headers, data=payload)
    data = json.loads(res.text)['data']
    soup = BeautifulSoup(data, 'html.parser')
    rows = soup.find_all('tr')

    schedule = {}
    current_date = ''

    for tr in rows:
        tds = tr.find_all('td')
        
        if len(tds) == 1:
            current_date = f'{tds[0].text}' # YYYY년 MM월 DD일 X요일
            schedule[current_date] = []
            continue
            
        elif len(tds) == 8:
            currency = tds[1].text.strip()
            country = '한국' if currency == 'KRW' else '미국'

            title = tds[3].text.strip()
            actual = tds[4].text.strip()
            forecast = tds[5].text.strip()
            previous = tds[6].text.strip()

            excluded_keywords = ['신규 실업수당청구건수', '주택판매', '원유재고', '성명서', '연설', '증언', 'FOMC', '경제전망', '회의록', '기자회견', '필라델피아', 'JOLTS', 'CB', 'ISM', 'ADP', '시간당 임금']
            if any(keyword in title for keyword in excluded_keywords):
                continue

            if title[:2] == country: # 제목에 국가명이 포함되어있는 경우가 있음
                event_text = f'{title} (실제: {actual} // 예상치: {forecast} // 이전: {previous})'
            else:
                event_text = f'{country} {title} (실제: {actual} // 예상치: {forecast} // 이전: {previous})'
            
            schedule[current_date].append(event_text)

    previous_business_day = get_previous_business_day()

    result_text = ''
    for date_str, events in schedule.items():
        if not events:
            continue

        date = datetime.strptime(date_str[:-4], "%Y년 %m월 %d일").date()
        if date < previous_business_day:
            continue

        weekday = date_str[-3]

        result_text += f'\n{date.month}월 {date.day}일 ({weekday})\n'
        result_text += '\n\n'.join(events) + '\n'

    return result_text.strip() or "해당없음"

In [8]:
from datetime import datetime

now = datetime.now()
date = f'{now.year}{now.month:02d}{now.day:02d}'
weekdays = ['월', '화', '수', '목', '금', '토', '일']

주요일정 = get_주요일정('thisWeek')
if now.weekday() == 4: # 금요일
    다음주_주요일정 = get_주요일정('nextWeek')
    주요일정 += f'\n\n\n● 다음주 주요일정 (인베스팅닷컴)\n{다음주_주요일정}'

투자경고종목 = get_투자경고종목(date)
투자위험종목 = get_투자위험종목(date)



장전문자 = f'''
안녕하세요, SK증권 Passive영업팀입니다.
{now.year}년 {now.month}월 {now.day}일 ({weekdays[now.weekday()]}) 장시작 전 주요 증시&공시정보 안내드립니다.

● 해외시장 주요지표 변동 (한국기준 전영업일 15:30 ~ 당일 08:00)
- 미국 S&P500선물 : %
- 미국 나스닥 100선물 : %
- 미국 원달러환율(USD/KRW) : %


● 이번주 주요일정 (인베스팅닷컴)
{주요일정}


● 투자경고/위험 종목
- 경고
{투자경고종목}

- 위험
{투자위험종목}


● KOSPI200, KOSDAQ150 종목 주요 공시정보


● 최신 지수운영 공지(KOSPI200 및 KOSDAQ150)
해당없음

● 지수운영공지 종료 전 이벤트
해당없음
'''

print(장전문자)


안녕하세요, SK증권 Passive영업팀입니다.
2025년 8월 6일 (수) 장시작 전 주요 증시&공시정보 안내드립니다.

● 해외시장 주요지표 변동 (한국기준 전영업일 15:30 ~ 당일 08:00)
- 미국 S&P500선물 : %
- 미국 나스닥 100선물 : %
- 미국 원달러환율(USD/KRW) : %


● 이번주 주요일정 (인베스팅닷컴)
8월 5일 (화)
한국 소비자물가지수 (MoM)  (7월) (실제: 0.2% // 예상치: 0.2% // 이전: 0.0%)

한국 소비자물가지수 (YoY)  (7월) (실제: 2.1% // 예상치: 2.1% // 이전: 2.2%)

미국 서비스 구매관리자지수  (7월) (실제: 55.7 // 예상치: 55.2 // 이전: 52.9)


● 투자경고/위험 종목
- 경고
씨어스테크놀로지, SNT에너지, 효성중공업, 비에이치아이, 태성, 올릭스, 로지스몬, 큐러블, 파멥신

- 위험
SGA


● KOSPI200, KOSDAQ150 종목 주요 공시정보


● 최신 지수운영 공지(KOSPI200 및 KOSDAQ150)
해당없음

● 지수운영공지 종료 전 이벤트
해당없음



In [9]:
today = datetime.today().date()
previous_business_day = get_previous_business_day()

for date in [today, previous_business_day]:
    print(f'================= {date} =================')
    df = get_공시(date)

    keywords = ["자기주식", "추가상장", "변경상장", "추가 상장", "변경 상장", "증자", "감자", "선택권", "CB전환", "소각", "처분", "취득", "배당", "주주명부폐쇄", "기준일", "투자경고", "투자위험", "투자유의", "정지", "단일가매매", "상장폐지", "신주인수권", "분할", "합병"]
    pattern = '|'.join(keywords)
    filtered_df = df[df['공시제목'].str.contains(pattern)]

    for _, row in filtered_df.iterrows():
        print(f"[{row['시간']}] {row['회사명']:<15} - {row['공시제목']} ({row['링크']})")
    print()

================= 2025-08-06 =================
[07:39] HD현대인프라코어       - 자기주식취득 신탁계약 해지 결정 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20250806000001&docno=&viewerhost=&viewerport=)

================= 2025-08-05 =================
[17:33] 엔켐              - [정정]제3자의 전환사채매수선택권 행사 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20250805000505&docno=&viewerhost=&viewerport=)
[17:27] 파트론             - 자기주식 취득 결정 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20250805000453&docno=&viewerhost=&viewerport=)
[17:18] 심텍              - 추가상장(국내사모 전환사채 전환) (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20250805000482&docno=&viewerhost=&viewerport=)
[17:14] 메리츠금융지주         - 주권 관련 사채권의 취득결정(자회사의 주요경영사항) (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20250805000463&docno=&viewerhost=&viewerport=)
[17:11] 포스코퓨처엠          - 추가상장(유상증자(주주배정후 실권주 일반공모)) (https://kind.krx.co.kr/common/disclsviewer.do?m